In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pathlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import PIL

In [ ]:
images = './Videos/frames'
images = list(pathlib.Path(images).glob('**/*'))

In [ ]:
files = []
for img in images:
    i = {
        'frame': int(str(img).split('\\')[-1].replace('.jpg','')),
        'path': img
    }
    files.append(i)

In [ ]:
files = sorted(files, key=lambda k: k['frame'])

In [ ]:
path = 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1'
detector = hub.load(path)

In [ ]:
def drawBoxes(img, boxes, scores, classes, accuracy=0.4, boxes_shaped=False, scaled=False, color=(255, 0, 0), box_thickness=5):
    
    imgCopy = img.copy()
    if boxes_shaped==False:
        boxes = np.array(boxes).reshape(boxes.shape[1:])

    count = 0
    for ymin, xmin, ymax, xmax in boxes:
        if scaled==False:
            xscaler, yscaler = img.shape[1],img.shape[0]
            ymin = math.ceil(ymin*yscaler)
            ymax = math.ceil(ymax*yscaler)
            xmin = math.ceil(xmin*xscaler)
            xmax = math.ceil(xmax*xscaler)
        if classes[count] == 1:
            if scores[count] > accuracy:
                start_point = (xmin,ymin)
                end_point = (xmax,ymax)
                                
                centerPoint = ((math.ceil((xmax-xmin)/2))+xmin,(math.ceil((ymax-ymin)/2)+ymin))
                xx,yy = centerPoint
                xx += 1
                yy += 1
                
                cp1 = (xx,yy)
                pointColor = (255,0,0)
                
                cv2.rectangle(imgCopy, start_point, end_point, color, box_thickness)
                cv2.rectangle(imgCopy, centerPoint, cp1, pointColor, 10)
        count += 1
    plt.figure(figsize=(10,7))
    plt.imshow(imgCopy)

In [ ]:
def saveObjects(img, boxes, output_path, scores, classes, frame_number, accuracy=0.4, boxes_shaped=False, scaled=False):

    if boxes_shaped==False:
        boxes = np.array(boxes).reshape(boxes.shape[1:])
        
    count = 0
    img_count = 0
    for ymin, xmin, ymax, xmax in boxes:
        # if the object is a person (class[count] == 1)
        if classes[count] == 1:
            # if the models confident beyond a threshold, i.e. 0.4
            if scores[count] > accuracy:
                # scale the data if needed by multiplying y coordniates 
                # by height of image and x coordinates by width of image
                if scaled==False:
                    xscaler, yscaler = img.shape[1],img.shape[0]
                    ymin = math.ceil(ymin*yscaler)
                    ymax = math.ceil(ymax*yscaler)
                    xmin = math.ceil(xmin*xscaler)
                    xmax = math.ceil(xmax*xscaler)
                
                
                centerPoint = '('+str((math.ceil((xmax-xmin)/2))+xmin)+'COMMA'+str((math.ceil((ymax-ymin)/2)+ymin))+')'
                sp = '('+str(xmin)+'COMMA'+str(ymin)+')'
                ep = '('+str(xmax)+'COMMA'+str(ymax)+')'
                obj = cv2.resize(cv2.cvtColor(img[ymin:ymax,xmin:xmax],cv2.COLOR_BGR2RGB),(250,500))        
                fullPath = output_path+'/'+frame_number+'_'+str(img_count)+'_'+centerPoint+'_'+sp+'_'+ep+'.jpg'
                cv2.imwrite(fullPath,obj)
                img_count += 1
        count += 1

In [ ]:
def detectPeople(imagePath, save=False, outputPath=None, display=True, accuracy=0.4):
    
    img = cv2.imread(imagePath)
    
    frame_number = imagePath.split('\\')[-1].replace('.jpg','')
    
    # Change to a RGB image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Reshape the image from (720, 1280, 3) to (1, 720, 1280, 3), which is needed to input into the model
    rsimg = img[np.newaxis, ...]
    
    #detect the objects
    detector_output = detector(rsimg)
    
    # Get an array of the different classes detected
    class_ids = detector_output["detection_classes"]
    classes = list(class_ids.numpy()[0])

    # Get an array of the confidence scores for each object
    d_scores = detector_output['detection_scores']
    scores = list(d_scores.numpy()[0])

    # Get the boxes that surround the objects
    d_boxes = detector_output["detection_boxes"]
    
    if display:
        drawBoxes(img,d_boxes, scores, classes, accuracy=accuracy, color=(0,255,0), box_thickness=2)
        
    if save:
        if outputPath == None:
            outputPath = './Videos/Objects'
            
        saveObjects(img, d_boxes, outputPath, scores, classes, frame_number, accuracy=accuracy)

In [ ]:
for i, file in enumerate(files):
    detectPeople(str(file['path']), save=True, display=False)
    
print('Objects have been saved')